## 1. Import Libraries

In [ ]:
# Core libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import random
from PIL import Image

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications import VGG16, ResNet50, MobileNetV2

# Evaluation
from sklearn.metrics import classification_report, confusion_matrix

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"Num GPUs Available: {len(tf.config.list_physical_devices('GPU'))}")

## 2. Data Exploration

In [ ]:
# Define paths
BASE_DIR = Path('data')
TRAIN_DIR = BASE_DIR / 'seg_train'
TEST_DIR = BASE_DIR / 'seg_test'
PRED_DIR = BASE_DIR / 'seg_pred'

# Class names
CLASS_NAMES = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
NUM_CLASSES = len(CLASS_NAMES)

print(f"Training directory: {TRAIN_DIR}")
print(f"Test directory: {TEST_DIR}")
print(f"Prediction directory: {PRED_DIR}")
print(f"\nClasses: {CLASS_NAMES}")

In [ ]:
# Count images in each class
def count_images(directory):
    """Count images in each class folder."""
    counts = {}
    for class_name in CLASS_NAMES:
        class_path = directory / class_name
        if class_path.exists():
            counts[class_name] = len(list(class_path.glob('*')))
        else:
            counts[class_name] = 0
    return counts

train_counts = count_images(TRAIN_DIR)
test_counts = count_images(TEST_DIR)

print("Training set distribution:")
for class_name, count in train_counts.items():
    print(f"  {class_name}: {count} images")
print(f"  Total: {sum(train_counts.values())} images")

print("\nTest set distribution:")
for class_name, count in test_counts.items():
    print(f"  {class_name}: {count} images")
print(f"  Total: {sum(test_counts.values())} images")

In [ ]:
# Visualize class distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Training set
axes[0].bar(train_counts.keys(), train_counts.values(), color='steelblue')
axes[0].set_title('Training Set Distribution', fontsize=14)
axes[0].set_xlabel('Class')
axes[0].set_ylabel('Number of Images')
axes[0].tick_params(axis='x', rotation=45)

# Test set
axes[1].bar(test_counts.keys(), test_counts.values(), color='coral')
axes[1].set_title('Test Set Distribution', fontsize=14)
axes[1].set_xlabel('Class')
axes[1].set_ylabel('Number of Images')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Display sample images from each class
def display_sample_images(directory, num_samples=3):
    """Display sample images from each class."""
    fig, axes = plt.subplots(NUM_CLASSES, num_samples, figsize=(12, 16))
    
    for i, class_name in enumerate(CLASS_NAMES):
        class_path = directory / class_name
        images = list(class_path.glob('*'))
        samples = random.sample(images, min(num_samples, len(images)))
        
        for j, img_path in enumerate(samples):
            img = Image.open(img_path)
            axes[i, j].imshow(img)
            axes[i, j].axis('off')
            if j == 0:
                axes[i, j].set_title(class_name.upper(), fontsize=12, fontweight='bold')
    
    plt.suptitle('Sample Images from Each Class', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

display_sample_images(TRAIN_DIR)

In [ ]:
# Check image dimensions
def analyze_image_dimensions(directory, sample_size=100):
    """Analyze image dimensions in the dataset."""
    widths, heights = [], []
    
    for class_name in CLASS_NAMES:
        class_path = directory / class_name
        images = list(class_path.glob('*'))[:sample_size]
        
        for img_path in images:
            try:
                img = Image.open(img_path)
                widths.append(img.size[0])
                heights.append(img.size[1])
            except:
                pass
    
    return widths, heights

widths, heights = analyze_image_dimensions(TRAIN_DIR)

print(f"Image dimensions analysis (sample):")
print(f"  Width  - Min: {min(widths)}, Max: {max(widths)}, Mean: {np.mean(widths):.1f}")
print(f"  Height - Min: {min(heights)}, Max: {max(heights)}, Mean: {np.mean(heights):.1f}")

## 3. Data Preprocessing

In [ ]:
# Hyperparameters
IMG_SIZE = 150
BATCH_SIZE = 32
EPOCHS = 30

print(f"Image Size: {IMG_SIZE}x{IMG_SIZE}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Epochs: {EPOCHS}")

In [ ]:
# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% for validation
)

# Only rescaling for test data
test_datagen = ImageDataGenerator(rescale=1./255)

print("Data generators created with augmentation for training.")

In [ ]:
# Create data generators
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

print(f"\nClass indices: {train_generator.class_indices}")

In [ ]:
# Visualize augmented images
def visualize_augmentation(generator, num_images=6):
    """Visualize augmented images."""
    batch = next(generator)
    images, labels = batch[0][:num_images], batch[1][:num_images]
    
    fig, axes = plt.subplots(2, 3, figsize=(12, 8))
    axes = axes.flatten()
    
    class_names_list = list(train_generator.class_indices.keys())
    
    for i, (img, label) in enumerate(zip(images, labels)):
        axes[i].imshow(img)
        class_idx = np.argmax(label)
        axes[i].set_title(f'Class: {class_names_list[class_idx]}')
        axes[i].axis('off')
    
    plt.suptitle('Augmented Training Images', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_augmentation(train_generator)

## 4. Model Architecture

We'll build two models:
1. A custom CNN from scratch
2. A transfer learning model using a pre-trained network

In [ ]:
# Custom CNN Model
def build_custom_cnn(input_shape=(IMG_SIZE, IMG_SIZE, 3), num_classes=NUM_CLASSES):
    """Build a custom CNN model."""
    model = models.Sequential([
        # First Conv Block
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Second Conv Block
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Third Conv Block
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Fourth Conv Block
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Fully Connected Layers
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

# Build and display model
custom_model = build_custom_cnn()
custom_model.summary()

In [ ]:
# Transfer Learning Model using MobileNetV2
def build_transfer_model(input_shape=(IMG_SIZE, IMG_SIZE, 3), num_classes=NUM_CLASSES):
    """Build a transfer learning model using MobileNetV2."""
    
    # Load pre-trained MobileNetV2
    base_model = MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    
    # Freeze base model layers
    base_model.trainable = False
    
    # Build model
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

# Build transfer learning model
transfer_model = build_transfer_model()
transfer_model.summary()

In [ ]:
# Select which model to train (change this to switch models)
MODEL_TYPE = 'transfer'  # 'custom' or 'transfer'

if MODEL_TYPE == 'custom':
    model = custom_model
    print("Using Custom CNN Model")
else:
    model = transfer_model
    print("Using Transfer Learning Model (MobileNetV2)")

In [ ]:
# Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("Model compiled successfully!")

## 5. Training

In [ ]:
# Define callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=3,
        min_lr=1e-7,
        verbose=1
    ),
    ModelCheckpoint(
        'best_model.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

print("Callbacks defined:")
print("  - EarlyStopping (patience=5)")
print("  - ReduceLROnPlateau (factor=0.2, patience=3)")
print("  - ModelCheckpoint (save best model)")

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# Plot training history
def plot_training_history(history):
    """Plot training and validation accuracy/loss."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Accuracy
    axes[0].plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
    axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
    axes[0].set_title('Model Accuracy', fontsize=14)
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Accuracy')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Loss
    axes[1].plot(history.history['loss'], label='Training Loss', linewidth=2)
    axes[1].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
    axes[1].set_title('Model Loss', fontsize=14)
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_training_history(history)

## 6. Evaluation

In [ ]:
# Load best model
best_model = keras.models.load_model('best_model.keras')
print("Best model loaded successfully!")

In [ ]:
# Evaluate on test set
test_loss, test_accuracy = best_model.evaluate(test_generator, verbose=1)
print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

In [ ]:
# Get predictions
test_generator.reset()
predictions = best_model.predict(test_generator, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

# Class names
class_names = list(test_generator.class_indices.keys())

print(f"Total predictions: {len(predicted_classes)}")
print(f"Total true labels: {len(true_classes)}")

In [ ]:
# Classification Report
print("Classification Report:")
print("=" * 60)
print(classification_report(true_classes, predicted_classes, target_names=class_names))

In [ ]:
# Confusion Matrix
cm = confusion_matrix(true_classes, predicted_classes)

plt.figure(figsize=(10, 8))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=class_names,
    yticklabels=class_names
)
plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.tight_layout()
plt.show()

In [ ]:
# Per-class accuracy
per_class_accuracy = cm.diagonal() / cm.sum(axis=1)

plt.figure(figsize=(10, 6))
bars = plt.bar(class_names, per_class_accuracy * 100, color='steelblue')
plt.axhline(y=test_accuracy * 100, color='red', linestyle='--', label=f'Overall Accuracy: {test_accuracy*100:.2f}%')
plt.title('Per-Class Accuracy', fontsize=14, fontweight='bold')
plt.xlabel('Class')
plt.ylabel('Accuracy (%)')
plt.xticks(rotation=45)
plt.legend()
plt.ylim(0, 100)

# Add value labels on bars
for bar, acc in zip(bars, per_class_accuracy):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
             f'{acc*100:.1f}%', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

## 7. Predictions

In [ ]:
# Function to predict a single image
def predict_image(model, image_path, class_names):
    """Predict class for a single image."""
    img = Image.open(image_path)
    img_resized = img.resize((IMG_SIZE, IMG_SIZE))
    img_array = np.array(img_resized) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    prediction = model.predict(img_array, verbose=0)
    predicted_class = class_names[np.argmax(prediction)]
    confidence = np.max(prediction) * 100
    
    return predicted_class, confidence, prediction[0]

# Display predictions with confidence
def display_predictions(model, directory, class_names, num_samples=9):
    """Display predictions for random test images."""
    all_images = []
    for class_name in class_names:
        class_path = Path(directory) / class_name
        all_images.extend(list(class_path.glob('*')))
    
    samples = random.sample(all_images, min(num_samples, len(all_images)))
    
    fig, axes = plt.subplots(3, 3, figsize=(12, 12))
    axes = axes.flatten()
    
    for i, img_path in enumerate(samples):
        true_class = img_path.parent.name
        predicted_class, confidence, _ = predict_image(model, img_path, class_names)
        
        img = Image.open(img_path)
        axes[i].imshow(img)
        
        color = 'green' if predicted_class == true_class else 'red'
        axes[i].set_title(f'True: {true_class}\nPred: {predicted_class} ({confidence:.1f}%)', 
                         color=color, fontsize=10)
        axes[i].axis('off')
    
    plt.suptitle('Model Predictions on Test Images', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

display_predictions(best_model, TEST_DIR, class_names)

In [ ]:
# Predict on images in seg_pred folder
def predict_folder(model, pred_dir, class_names):
    """Make predictions on all images in a folder."""
    pred_path = Path(pred_dir)
    
    if not pred_path.exists():
        print(f"Prediction folder {pred_dir} does not exist.")
        return
    
    images = list(pred_path.glob('*'))
    image_files = [f for f in images if f.suffix.lower() in ['.jpg', '.jpeg', '.png']]
    
    if not image_files:
        print("No images found in prediction folder.")
        return
    
    results = []
    for img_path in image_files:
        predicted_class, confidence, probs = predict_image(model, img_path, class_names)
        results.append({
            'filename': img_path.name,
            'predicted_class': predicted_class,
            'confidence': confidence
        })
    
    results_df = pd.DataFrame(results)
    return results_df

# Predict on seg_pred folder
pred_results = predict_folder(best_model, PRED_DIR, class_names)
if pred_results is not None and len(pred_results) > 0:
    print("\nPrediction Results:")
    print(pred_results.to_string(index=False))

In [ ]:
# Save the model
model.save('intel_image_classifier.keras')
print("Model saved as 'intel_image_classifier.keras'")

## Summary

In this notebook, we:

1. **Explored the dataset** - Analyzed the distribution of images across 6 classes
2. **Preprocessed the data** - Applied data augmentation and normalization
3. **Built models** - Created both a custom CNN and a transfer learning model
4. **Trained the model** - Used callbacks for early stopping and learning rate reduction
5. **Evaluated performance** - Generated confusion matrix and classification report
6. **Made predictions** - Demonstrated how to use the model for inference

### Next Steps
- Fine-tune the transfer learning model by unfreezing some layers
- Experiment with different architectures (ResNet, EfficientNet)
- Try different hyperparameters and augmentation strategies
- Implement cross-validation for more robust evaluation
- Deploy the model as a web service or mobile app